In [27]:
# Add any additional libraries you will use for this assignment in this block
import pandas as pd
import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt 

In [28]:
Df = pd.read_csv('GolfDataset.csv')
Df

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [29]:
Tbl = pd.crosstab(index = Df['Outlook'], columns = Df['Play Golf'])
Tbl

Play Golf,No,Yes
Outlook,,
Overcast,0,4
Rainy,3,2
Sunny,2,3


In [30]:
rYes = Tbl.loc['Rainy']['Yes']
rNo = Tbl.loc['Rainy']['No']

print(str("The number of days the Outlook was Rainy and the person played golf is: " + str(rYes)))
print(str("The number of days it Outlook was Rainy and the person did not play golf is: " + str(rNo)))
print("\n")

print(str("The proability of playing golf given it is rainy"+ '\n'+ \
          "p(Playing Golf = Yes|Outlook = 'Rainy') is: " + str(rYes/(rYes+rNo))))

print("\n")


print(str("The proability of not playing golf given it is rainy"+ '\n'+ \
          "p(Playing Golf = No|Outlook = 'Rainy') is: " + str(rNo/(rYes+rNo))))

The number of days the Outlook was Rainy and the person played golf is: 2
The number of days it Outlook was Rainy and the person did not play golf is: 3


The proability of playing golf given it is rainy
p(Playing Golf = Yes|Outlook = 'Rainy') is: 0.4


The proability of not playing golf given it is rainy
p(Playing Golf = No|Outlook = 'Rainy') is: 0.6


<div class="alert alert-block alert-danger">
Write a function to compute Gini Index of a given split.
</div>

Note that Gini index of a feature is given by: 

$$ Gini_{split}(\mathrm{feature})  = \sum_{i = 1}^k \frac{n_i}{n} Gini(\mathrm{feature} = t_i)$$

where,

$n_i$ is the number of records at child $i$

$n$ is the number of records at parent $p$

$$ Gini(\mathrm{feature} = t_i)  = 1 - \sum_{c \in C} p(\mathrm{class} = c | \mathrm{feature} = t_i)^2$$

where,

$C$ is the set of classes


As indicated below, your function should accept a dataframe similar to the contingency `Tbl` defined above. I recommend writing two functions, one for computing $Gini(\mathrm{feature} = t_i)$ which you can use to compute $Gini_{split}(\mathrm{feature})$.

In [31]:
def computeGiniFeature(TblRow):
    # Note that TblRow should be a list or 1-dimensional numpy array obtained using Tbl.loc[ix].values
    # Your code here
    giniFeature=0
    for i in range(len(TblRow)):
        p=TblRow[i]/sum(TblRow)
        giniFeature += p**2
    giniFeature = 1-giniFeature

    return(giniFeature)


def computeGiniSplit(Tbl):
    
    # Initialize additional variables if necessary
    giniSplit = 0
    for ix, row in Tbl.iterrows():
        # You can call the computeGiniFeature(Tbl.loc[ix]) here for every row in Tbl  
        # and combine the values to obtain the final giniSplit
        TblRow = Tbl.loc[ix].values # Try not to change this part of the code
        giniFeature = computeGiniFeature(TblRow)
        # Your code here
        a= TblRow.sum()
        b=Tbl.sum().sum()
        giniSplit += a/b*giniFeature
        
    return(giniSplit)

<div class="alert alert-block alert-danger">
Write a function to compute Entropy of a given split. 
</div>

Note that Entropy of a feature is given by: 

$$ Entropy_{split}(\mathrm{feature})  = \sum_{i = 1}^k \frac{n_i}{n} Entropy(\mathrm{feature} = t_i)$$

where,

$n_i$ is the number of records at child $i$

$n$ is the number of records at parent $p$

$$ Entropy(\mathrm{feature} = t_i)  =  - \sum_{c \in C} p(c | t_i)\log_2 p(c | t_i)$$

where,

$C$ is the set of classes


As indicated below, your function should accept a dataframe similar to the contingency `Tbl` defined above. I recommend writing two functions, one for computing $Entropy(\mathrm{feature} = t_i)$ which you can use to compute $Entropy_{split}(\mathrm{feature})$.

In [32]:
def computeEntropyFeature(TblRow):
    entropyFeature=0
    for i in range(len(TblRow)):
        p=TblRow[i]/sum(TblRow)
        if p==0:
            continue
        entropyFeature -= p*np.log2(p)
    return(entropyFeature)

def computeEntropySplit(Tbl):
    
    # Initialize additional variables if necessary
    entropySplit = 0
    for ix, row in Tbl.iterrows():
        # You can call the computeEntropyFeature(Tbl.loc[ix]) here for every row in Tbl  
        # and combine the values to obtain the final entropySplit
        TblRow = Tbl.loc[ix].values # Try not to change this part of the code
        entropyFeature = computeEntropyFeature(TblRow)
        # Your code here
        a= TblRow.sum()
        b=Tbl.sum().sum()
        entropySplit += a/b*entropyFeature
        
    return(entropySplit)

<div class="alert alert-block alert-danger">
To identify the root node for the decision tree, based on the information gain computed.
</div>

$$ Information Gain(\textrm{feature} = f)  = P - M(f)$$

where,

 $P$ is the Gini index before split, and
 
 $ M(f)$  is the Gini Index of the feature f
 
Note that since computeGiniFeature() accepts a list of values, you should be able to compute $P$ using computeGiniFeature() function by passing the appropriate list as input. Same goes for computing InformationGain using Entropy as impurity measure.

In [33]:
#%%    
tab_label= Df['Play Golf'].value_counts()
P=computeGiniFeature(tab_label)
M=[]
for i in range(len(Df.columns)-1):
    Tbl = pd.crosstab(index = Df.iloc[:,i], columns = Df.iloc[:,-1])
    M.append(computeGiniSplit(Tbl))
    print(str("Information gain of Gini for " + Df.columns[i] +" as root node is: " + str(P-M[i])))
    print("\n")
IG= []
IG= P-M    
maxcol= IG.argmax()
print(str(Df.columns.values[maxcol] + " with max.IG shall be picked as root node"))
print('***************************************************')
print("\n")

#%%
P=computeEntropyFeature(tab_label)
M=[]
for i in range(len(Df.columns)-1):
    Tbl = pd.crosstab(index = Df.iloc[:,i], columns = Df.iloc[:,-1])
    M.append(computeEntropySplit(Tbl))
    print(str("Information gain of Entropy for " + Df.columns[i] +" as root node is: " + str(P-M[i])))
    print("\n")
IG= []
IG= P-M 
maxcol= IG.argmax()
print(str(Df.columns.values[maxcol] + " with max.IG shall be picked as root node"))
print('***************************************************')
print("\n")

Information gain of Gini for Outlook as root node is: 0.11632653061224485


Information gain of Gini for Temp as root node is: 0.018707482993197244


Information gain of Gini for Humidity as root node is: 0.09183673469387743


Information gain of Gini for Windy as root node is: 0.030612244897959162


Outlook with max.IG shall be picked as root node
***************************************************


Information gain of Entropy for Outlook as root node is: 0.24674981977443933


Information gain of Entropy for Temp as root node is: 0.02922256565895487


Information gain of Entropy for Humidity as root node is: 0.15183550136234159


Information gain of Entropy for Windy as root node is: 0.04812703040826949


Outlook with max.IG shall be picked as root node
***************************************************




<div class="alert alert-block alert-danger">
Computing the InformationGain using Gini index as the impurity measure. 
</div>


In [34]:
#%%
Df_out = Df['Outlook'].unique()
for j in range(len(Df_out)):
    Df_out_split = (Df[Df['Outlook']== Df_out[j]])
    Df_out_split = Df_out_split.drop(columns=['Outlook'])
    
    if len(Df_out_split['Play Golf'].unique()) > 1:
        
        tab_label= Df_out_split['Play Golf'].value_counts()
        P=computeGiniFeature(tab_label)
        M=[]
        for i in range(len(Df_out_split.columns)-1):
            Tbl = pd.crosstab(index = Df_out_split.iloc[:,i], columns = Df_out_split.iloc[:,-1])
            M.append(computeGiniSplit(Tbl))
            print(str("Information gain of Gini for " + Df_out_split.columns[i] + \
                      " as child of " + Df_out[j] + " is: " + str(P-M[i])))
            print("\n")
        IG= []
        IG= P-M    
        maxcol= IG.argmax()
        print(str(Df.columns.values[maxcol] + " with max.IG shall be picked as child of " + Df_out[j]))
        print('***************************************************')
        print("\n")
        
    else:
        print(str(Df_out[j] + " is a leaf node "))
        print('***************************************************')
        print("\n")

Information gain of Gini for Temp as child of Rainy is: 0.27999999999999997


Information gain of Gini for Humidity as child of Rainy is: 0.48


Information gain of Gini for Windy as child of Rainy is: 0.013333333333333308


Temp with max.IG shall be picked as child of Rainy
***************************************************


Overcast is a leaf node 
***************************************************


Information gain of Gini for Temp as child of Sunny is: 0.013333333333333308


Information gain of Gini for Humidity as child of Sunny is: 0.013333333333333308


Information gain of Gini for Windy as child of Sunny is: 0.48


Humidity with max.IG shall be picked as child of Sunny
***************************************************




In [37]:
#Based on the data provided in `ForestTypes/training.csv`. How may features and classes are in this dataset?
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Df = pd.read_csv('ForestTypes/training.csv')
print(Df.shape)
print('27 features and 4 classes')
Df.iloc[:,0].value_counts()


(198, 28)
27 features and 4 classes


s     59
d     54
h     48
o     37
Name: class, dtype: int64

In [38]:
#Using data in `ForestTypes/training.csv` build a Logisitc Regression Classifier.

from sklearn.linear_model import LogisticRegression, LinearRegression
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
# The following line ensures our plots appear in-line, 
# as opposed to appearing in a new window.
%matplotlib inline 
import matplotlib.pyplot as plt 



In [39]:
#
Df_training = pd.read_csv('ForestTypes/training.csv')
Df_training.shape
Df_training.iloc[:,0].value_counts()

Df_testing = pd.read_csv('ForestTypes/testing.csv')
Df_testing.shape
Df_testing.iloc[:,0].value_counts()

predictors = Df_training.columns[1:]
X_train = Df_training.loc[:,predictors]
X_test = Df_testing.loc[:,predictors]
y_train = Df_training.iloc[:,0]
le =LabelEncoder() # Initilaize Data Encoder
le.fit(y_train) # Fit encoder to data
y_train = le.transform(y_train)
y_test = Df_testing.iloc[:,0]
le.fit(y_test)
y_test = le.transform(y_test)



In [41]:
sklearn_logistic = LogisticRegression(solver='liblinear')
sklearn_logistic.fit(X_train, y_train)
#print(sklearn_logistic.intercept_, sklearn_logistic.coef_)
sklearn_predict = sklearn_logistic.predict(X_test)
pd.Series(sklearn_predict).value_counts()


from sklearn import metrics
Accuracy = metrics.accuracy_score(y_test, sklearn_predict)
print('The accuracy is %.2f%%:' %(Accuracy*100))

The accuracy is 85.23%:



<div class="alert alert-block alert-danger">
Using data in `ForestTypes/training.csv` build a SVM with a polynomial kernel. Perform 5-fold cross validation on the training data to identify the best choice of parameters C and degree. Try the following values for parameter search:
    
   - Penalty parameter C of the error term: C = 1, 5, 10
   - Degree of the polynomial kernel function (‘poly’): degree = 2, 3
</div>

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# The following line ensures our plots appear in-line, 
# as opposed to appearing in a new window.
%matplotlib inline 
import matplotlib.pyplot as plt 

In [43]:
Df_training = pd.read_csv('ForestTypes/training.csv')
Df_training.shape
Df_training.iloc[:,0].value_counts()
Df_testing = pd.read_csv('ForestTypes/testing.csv')
Df_testing.shape
Df_testing.iloc[:,0].value_counts()


predictors = Df_training.columns[1:]
X_train = Df_training.loc[:,predictors]
X_test = Df_testing.loc[:,predictors]

y_train = Df_training.iloc[:,0]
le =LabelEncoder() # Initilaize Data Encoder
le.fit(y_train) # Fit encoder to data
y_train = le.transform(y_train)
y_test = Df_testing.iloc[:,0]
le.fit(y_test)
y_test = le.transform(y_test)

In [44]:
parameters = [
              {'C': [1, 5, 10], 'degree': [2, 3], 'kernel': ['poly']},
              ]
grid_svc = GridSearchCV(estimator = SVC(),param_grid =parameters,scoring='accuracy',cv=5,verbose =1)

grid_svc.fit(X_train,y_train)

grid_svc.best_params_, grid_svc.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


({'C': 10, 'degree': 3, 'kernel': 'poly'}, 0.9698717948717949)

In [45]:
pred_svc = grid_svc.predict(X_test)
Accuracy = metrics.accuracy_score(y_test,pred_svc)
print('The accuracy is %.2f%%:' %(Accuracy*100))

The accuracy is 84.31%:


Based on the grid search cross validation, the C = 10, degree = 3.

It is compared with the original paper that published this data here: http://www.cr.chiba-u.jp/~tateishi-lab/pdf/2011/Using%20geographically%202011.pdf .
The paper is using the RBF kernel, with C=1010, $\gamma$ = 0.003 and 1.95 for 27 and 9 variables.
The accuracies are 82.2\% for 9 variables and 85.9\% for 27 variables.
Compared with results here 84.3\% using polynominal kernel.